# Decorator

A decorator is a function that modifies or extends the behavior of another function (or method) without changing its source code.

It’s commonly used for:

* Logging
* Access control
* Memoization/caching
* Timing functions
* Input validation

# Logging Decorator

In [1]:
from datetime import datetime

In [2]:
def log_function_call(func):
    def wrapper(*args, **kwargs):
        print(f"[{datetime.now()}] Calling {func.__name__} with {args}, {kwargs}")
        result = func(*args, **kwargs)
        print(f"[{datetime.now()}] {func.__name__} returned {result}")
        return result
    return wrapper

@log_function_call
def add(a, b):
    return a + b

In [3]:
add(2,3)

[2025-06-07 21:39:27.149430] Calling add with (2, 3), {}
[2025-06-07 21:39:27.149533] add returned 5


5

# Async Function Decorators

In [11]:
import asyncio
import functools
import nest_asyncio

nest_asyncio.apply()

def async_logger(func):
    @functools.wraps(func)
    async def wrapper(*args, **kwargs):
        print(f"[Async Log] Calling {func.__name__}")
        result = await func(*args, **kwargs)
        print(f"[Async Log] Done: {result}")
        return result
    return wrapper

@async_logger
async def slow_add(a, b):
    await asyncio.sleep(1)
    return a + b

async def main():
    res = await asyncio.gather(slow_add(1, 2), slow_add(4, 5))
    return res

# asyncio.run(slow_add(1, 2))
loop = asyncio.get_event_loop()
result = loop.run_until_complete(main())
print(result)


[Async Log] Calling slow_add
[Async Log] Calling slow_add
[Async Log] Done: 3
[Async Log] Done: 9
[3, 9]


Using `@functools.wraps(func)` ensures that metadata like `__name__`, `__doc__`, etc. are preserved in the decorated function.

# Decorators for Class Methods

In [ ]:
import functools

def log_method(func):
    @functools.wraps(func)
    def wrapper(self, *args, **kwargs):
        print(f"[LOG] {func.__name__} called with args={args}, kwargs={kwargs}")
        result = func(self, *args, **kwargs)
        print(f"[LOG] {func.__name__} returned {result}")
        return result
    return wrapper

class Calculator:
    def __init__(self, name):
        self.name = name

    @log_method
    def multiply(self, a, b):
        return a * b

calc = Calculator("Basic")
calc.multiply(4, 5)


# Exception Handling Decorator

In [ ]:
import functools
import traceback

def catch_errors(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(f"[ERROR] {func.__name__} raised {type(e).__name__}: {e}")
            traceback.print_exc()
            return None  # Or raise, or a fallback value
    return wrapper

@catch_errors
def risky_division(a, b):
    return a / b

print(risky_division(10, 2))   # ✅ Works
print(risky_division(10, 0))   # ❌ Caught


# Fully featured decorator

Create a fully featured decorator that works with:

* ✅ Datetime logging

* ✅ Class methods (self support)

* ✅ Exception handling

* ✅ Error logging to a file

In [13]:
import functools
from datetime import datetime
import traceback

def log_and_handle(logfile='errors.log'):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            func_name = func.__name__
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print(f"[{timestamp}] Calling {func_name} with args={args}, kwargs={kwargs}")

            try:
                result = func(*args, **kwargs)
                print(f"[{timestamp}] {func_name} returned: {result}")
                return result
            except Exception as e:
                error_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                error_message = (
                    f"[{error_time}] ERROR in {func_name}:\n"
                    f"Args: {args}\n"
                    f"Kwargs: {kwargs}\n"
                    f"{type(e).__name__}: {str(e)}\n"
                    f"{traceback.format_exc()}\n"
                    + "-"*50 + "\n"
                )
                print(f"[{error_time}] {func_name} raised an error: {e}")
                with open(logfile, 'a') as f:
                    f.write(error_message)
                return None  # or raise if preferred
        return wrapper
    return decorator


In [14]:
class Demo:
    def __init__(self, name):
        self.name = name

    @log_and_handle()
    def greet(self, other_name):
        return f"Hello {other_name}, I'm {self.name}!"

    @log_and_handle()
    def divide(self, x, y):
        return x / y

# Use the class
d = Demo("GPT")
d.greet("Alice")
d.divide(10, 2)
d.divide(10, 0)  # Will trigger and log a ZeroDivisionError


[2025-06-07 21:59:44] Calling greet with args=(<__main__.Demo object at 0x105ce36a0>, 'Alice'), kwargs={}
[2025-06-07 21:59:44] greet returned: Hello Alice, I'm GPT!
[2025-06-07 21:59:44] Calling divide with args=(<__main__.Demo object at 0x105ce36a0>, 10, 2), kwargs={}
[2025-06-07 21:59:44] divide returned: 5.0
[2025-06-07 21:59:44] Calling divide with args=(<__main__.Demo object at 0x105ce36a0>, 10, 0), kwargs={}
[2025-06-07 21:59:44] divide raised an error: division by zero
